# problema di trasporto

Una società di logistica movimenta container vuoti da M magazzini a P porti

La richiesta dei porti (vedi tabella) può essere soddisfatta prelevando i container vuoti da uno qualsiasi dei magazzini. 

![tab1](./tab1.png)

Ogni magazzino, tuttavia, ha una disponibilità limitata di container (vedi tabella)

![tab2](./tab2.png)

La movimentazione avviene attraverso una flotta di camion, ognuno dei quali può trasportare al massimo 2 container. 

Il costo di ogni viaggio dipende dalla distanza che intercorre tra magazzino e porto ed è descritto dalla seguente tabella:

![tab3](./tab3.png)

Come devono essere organizzate le consegne al fine di minimizzare il costo totale di movimentazione?


## modello di programmazione matematica

![img1](./img1.png)

![img2](./img2.png)

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

nomi_magazzini = [ 'La Spezia', 'Trieste', 'Ancona', 'Napoli', 'Bari' ]
disp_magazzini = [ 20, 15, 25, 33, 21 ]
assert len(nomi_magazzini) == len(disp_magazzini)

nomi_porti = [ 'Padova', 'Arezzo', 'Roma', 'Teramo', 'Lecce', 'Catanzaro' ]
rich_porti = [ 10, 12, 20, 24, 18, 40 ]
assert len(nomi_porti) == len(rich_porti)

costi_list = [[8700, 3450, 10650, 21450, 24300],
          [11400, 10200, 4950, 11400, 18300], 
          [15150, 15900, 8550, 6600, 13500],
          [19650, 13500, 4650, 7200, 9450], 
          [30300, 25200, 16500, 9150, 2850], 
          [32160, 32910, 22410, 11160, 9990]]

model.i = RangeSet(0,len(nomi_magazzini)-1)
model.j = RangeSet(0,len(nomi_porti)-1)

costi = {}
for i in model.i:
    for j in model.j:
        costi[j, i] = costi_list[j][i]

model.m = Set(initialize=disp_magazzini)
model.r = Set(initialize=rich_porti)

model.c = Param(model.j, model.i, initialize=costi)

model.x = Var(model.j, model.i, domain=NonNegativeIntegers, initialize=0)
model.y = Var(model.j, model.i, domain=NonNegativeIntegers, initialize=0)

obj_expr = sum(sum(model.c[j, i]*model.y[j, i] for i in model.i) for j in model.j)

model.cost = Objective(expr = obj_expr, sense=minimize)

model.constraints = ConstraintList()

for i in model.i:
    model.constraints.add(expr = sum(model.x[j, i] for j in model.j) <= disp_magazzini[i])

for j in model.j:
    model.constraints.add(expr = sum(model.x[j, i] for i in model.i) >= rich_porti[j])

for j in model.j:
    for i in model.i:
        model.constraints.add(expr = 2*model.y[j, i] >= model.x[j, i])

#result = SolverFactory('glpk').solve(model, tee=True).write()

results = SolverFactory('glpk').solve(model, tee=True)

print ("The solver returned a status of: "+str(results.solver.status))

#model.pprint()
model.display() 

for v in model.component_data_objects(Var, active=True):
    print(v, value(v)) 

print(value(model.cost))

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpx7h4prl7.glpk.raw --wglp /tmp/tmpyp29svq3.glpk.glp --cpxlp
 /tmp/tmpr98sj78a.pyomo.lp
Reading problem data from '/tmp/tmpr98sj78a.pyomo.lp'...
42 rows, 61 columns, 121 non-zeros
60 integer variables, none of which are binary
406 lines were read
Writing problem data to '/tmp/tmpyp29svq3.glpk.glp'...
360 lines were written
GLPK Integer Optimizer, v4.65
42 rows, 61 columns, 121 non-zeros
60 integer variables, none of which are binary
Preprocessing...
41 rows, 60 columns, 120 non-zeros
60 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 41
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
41 rows, 60 columns, 120 non-zeros
      0: obj =   0.000000000e+00 inf =   1.240e+02 (6)
     40: obj =   5.448900000e+05 inf =   1.000e+01